<h1> Day 10 -CNN- MNIST handwritten image classification using VGG16- Transfer Learning</h1>

Importing the Essential modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical , plot_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models,layers
from tensorflow import keras


Loading the MNIST Data

In [ ]:
(x_train,y_train),(x_test,y_test)=keras.datasets.mnist.load_data()


11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(y_train)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
[5 0 4 ... 5 6 8]


Preprocessing the data to make it compatible  with the VGG16 model


In [ ]:
x_train=np.dstack([x_train]*3)
x_test=np.dstack([x_test]*3)
x_train.shape,x_test.shape

((60000, 28, 84), (10000, 28, 84))

Reshape images as per the tensor format required by tensorflow


In [ ]:
x_train=x_train.reshape(-1,28,28,3)
x_test=x_test.reshape(-1,28,28,3)
x_train.shape,x_test.shape

((60000, 28, 28, 3), (10000, 28, 28, 3))

Resizing the images into 48*48 as required for VGG16

In [ ]:
from keras.preprocessing.image import img_to_array, array_to_img

x_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in x_train])
x_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in x_test])
x_train.shape, x_test.shape

((60000, 48, 48, 3), (10000, 48, 48, 3))

Preparing the Data for the VGG16 model

In [ ]:
x=[]
x=x_train
y=[]
y=y_train

y=to_categorical(y)
x=np.array(x)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=5)

Defing the Model using Transfer learning

In [ ]:
from tensorflow.keras.applications import VGG16
model_vgg16=VGG16(weights='imagenet')



553467096/553467096 [==============================] - 6s 0us/step


Adding the input layer to the model

In [ ]:
input_layer=layers.Input(shape=(48,48,3))
model_vgg16=VGG16(weights='imagenet',input_tensor=input_layer,include_top=False)



58889256/58889256 [==============================] - 0s 0us/step


Adding the output layers

In [ ]:
last_layer=model_vgg16.output
flatten=layers.Flatten()(last_layer)
dense1=layers.Dense(100,activation='relu')(flatten)
dense1=layers.Dense(100,activation='relu')(flatten)
dense1=layers.Dense(100,activation='relu')(flatten)
output_layer=layers.Dense(10,activation='softmax')(flatten)

model=models.Model(inputs=input_layer,outputs=output_layer)


We are making all the layers intrainable except the last layer.

In [ ]:
for layer in model.layers[:-1]:
    layer.trainable=False


compiling the model


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

print("Model compilation completed.")
model.summary()

Model compilation completed.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (No

Model fitting

In [ ]:
history = model.fit(x_train,y_train,epochs=20,batch_size=128,verbose=True,validation_data=(x_test,y_test))

Evaluating the model

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)


In [ ]:
print("The loss is ",loss,"And the accuracy is ",accuracy)